In [4]:
%%capture
!pip install audiomentations pyroomacoustics pyloudnorm pydub

In [ ]:
%%capture
!pip install datasets

In [1]:
from datasets import load_dataset

In [2]:
timit = load_dataset("timit_asr")

Generating train split:   0%|          | 0/4620 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1680 [00:00<?, ? examples/s]

Dataset timit_asr downloaded and prepared to /root/.cache/huggingface/datasets/timit_asr/clean/2.0.1/31aa396113a9e0aaf4a6ba07260fd65ff80dfa0d4227fa76a27f1263725ac1b1. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
timit["train"][0]["audio"]["array"]

array([-2.1362305e-04,  6.1035156e-05,  3.0517578e-05, ...,
       -3.0517578e-05, -9.1552734e-05, -6.1035156e-05], dtype=float32)

In [7]:
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift
import numpy as np

augment = Compose([
    AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=0.5),
    TimeStretch(min_rate=0.8, max_rate=1.25, p=0.5),
    PitchShift(min_semitones=-4, max_semitones=4, p=0.5),
    Shift(min_fraction=-0.5, max_fraction=0.5, p=0.5),
])

# Generate 2 seconds of dummy audio for the sake of example
samples = timit["train"][0]["audio"]["array"]

# Augment/transform/perturb the audio data
augmented_samples = augment(samples=samples, sample_rate=16000)

In [8]:
import IPython.display as ipd
sr=16000
ipd.Audio(timit["train"][0]["audio"]["array"], rate=sr, autoplay=True)

In [9]:
ipd.Audio(augmented_samples, rate=sr, autoplay=True)